In [ ]:
%pip install vosk
%apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

In [ ]:
%pip install pyaudio

In [4]:
from vosk import Model, KaldiRecognizer
import sys
import json
import os
import time
import wave
import IPython
import pyaudio

Модель на vosk быстрая и даёт качественный результат (хотя некоторые слова могут быть чуть хуже, чем у модели с HF)
'vosk-model-ru-0.42' даёт хорошие результаты, но весит многовато
'vosk-model-small-ru-0.22' очень мало весит, но есть небольшие косяки в результатах
Можно покопаться в моделях и выбрать лучшее здесь: https://alphacephei.com/vosk/models

**ВАЖНО**: файл аудио для этой модели должен быть строго **моно** и **sampling rate файла = 16000 (16kHz)**

Запуск из файла

In [5]:
def speech_2_text(path, model):

    wf = wave.open(path, "rb")
    rec = KaldiRecognizer(model, 16000)

    result = ''
    last_n = False

    rec.SetWords(True)
    rec.SetPartialWords(True)

    while True:
        data = wf.readframes(8000)

        if len(data) == 0:
            break

        if rec.AcceptWaveform(data):
            res = json.loads(rec.Result())

            if res['text'] != '':
                result += f" {res['text']}"
                last_n = False

            elif not last_n:
                result += '\n'
                last_n = True

    res = json.loads(rec.FinalResult())
    result += f" {res['text']}"

    return result

In [6]:
path = "audio_samples/sample_1_mono_16khz.wav"

IPython.display.Audio(path)

In [9]:
model = Model(model_name="vosk-model-ru-0.42") # "/vosk-model-small-ru-0.52" - эта модель должна быть быстрее, но почему-то не заводится...

vosk-model-ru-0.42.zip: 100%|██████████| 1.80G/1.80G [03:52<00:00, 8.35MB/s]   


In [10]:
result = speech_2_text(path=path, model=model)
result

' помощник машиниста электровоза чему вы научитесь проводить техническое обслуживание приёмку и сдачу локомотива выполнять вспомогательные работы по управлению локомотивом и ведению поезда понимать причины возникновения неисправностей и умение их исправлять'

Работа "на лету" с микрофона

In [ ]:
#model = Model(model_name="vosk-model-ru-0.42")
rec = KaldiRecognizer(model, 16000)
p = pyaudio.PyAudio()
stream = p.open(
    format=pyaudio.paInt16,
    channels=1,
    rate=16000,
    input=True,
    frames_per_buffer=8000
)
stream.start_stream()
print('Talk now')

while True:
    data = stream.read(100)
    if len(data) == 0:
        break

    #print(rec.Result() if rec.AcceptWaveform(data) else rec.PartialResult())

print(rec.FinalResult())

In [26]:
res = json.loads(rec.FinalResult())['text']
res


''

In [ ]:
def speech_recognition(duration_seconds):
    rec = KaldiRecognizer(model, 16000)

    p = pyaudio.PyAudio()
    stream = p.open(
        format=pyaudio.paInt16,
        channels=1,
        rate=16000,
        input=True,
        frames_per_buffer=8000
    )
    stream.start_stream()
    print('Talk now')

    try:
        for _ in range(int(16000 / 100 * duration_seconds)):
            data = stream.read(100)
            if len(data) == 0:
                break
            
            cur_words = json.loads(rec.Result())['text'] if rec.AcceptWaveform(data) else json.loads(rec.PartialResult())['partial']
            if cur_words != '':
                print(cur_words)


    except KeyboardInterrupt:
        pass

    finally:
        stream.stop_stream()
        stream.close()
        p.terminate()

    return json.loads(rec.FinalResult())


In [ ]:
duration_seconds = 4  # Задайте требуемое время записи в секундах
result = speech_recognition(duration_seconds)
print("-" * 100)
print(f"Result text: {result['text']}")